In [1]:
import numpy as np
import wandb
from tqdm import trange
from util.mnist.tools import *
from util.pickleInterface import *
from util.whatwhere.encoder import *
from util.whatwhere.decoder import *
from util.whatwhere.noise import *
from util.willshaw.memory import *
from util.willshaw.plot import *
from util.pytorch.tools import np_to_grid
from util.distribution import *
from util.basic_utils import mse_detailed
from util.whatwhere.description_encoding import *
from util.whatwhere.generate import *


In [2]:
""" Code generation parameters """
rng = np.random.RandomState(0)  # reproducible
K = 20
Q = 21
n_epochs = 5
b = 0.8
wta = True
Fs = 2
T_what = 0.75

""" load mnist """
imgs, lbls, tst_imgs, tst_lbls = read_mnist(n_train=60000)
I = imgs.shape[1]
J = imgs.shape[2]

""" generate codes """
features = compute_features(imgs, lbls, K, Fs, rng, n_epochs, b, verbose=False)
codes, polar_params = compute_codes(
    imgs,
    K,
    Q,
    features,
    T_what,
    wta,
    n_epochs,
    b,
    Fs,
    verbose=False,
    set="trn"
)

code_size = codes.shape[1]


' tst_codes, tst_polar_params = compute_codes(\n    tst_imgs,\n    K,\n    Q,\n    features,\n    T_what,\n    wta,\n    n_epochs,\n    b,\n    Fs,\n    verbose=False,\n    set="tst"\n) '

In [4]:
wn = AAWN(code_size)

In [5]:
wn.store(codes)

In [6]:
gen_lbls = create_gen_lbls(n_classes=10, n_exs=3, transpose=False)

In [9]:
prob_dists = compute_dists(codes, lbls)
gen_codes = sample_from_dists(prob_dists, gen_lbls)

In [12]:
noise_type = "zero"
l_prob = [0, 0.25, 0.5, 0.75]

In [ ]:
for prob in l_prob:
    gen_codes_noisy = add_noise(gen_codes, noise_type, prob)
    ret = wn.retrieve(gen_codes_noisy)
    recons = recon_no_polar(ret, features, Q, K)